In [18]:
from fuzzywuzzy import fuzz, process
import os

C:\Users\373659\AppData\Local\Continuum\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
strike_folder = os.path.normpath('L:\BUDGET\I-Team\Data Library\LAUSD_strike')
os.chdir(strike_folder)

In [551]:
facility_polygon = pd.read_excel('Recreation_and_Parks_polygon_file.xlsx')
facility_polygon['Rec Center'] = np.where(facility_polygon['Name'].str.upper().str.contains('RECREATION CENTER'),1,0)
facility_polygon['Not a Dog Park'] = np.where(facility_polygon['Name'].str.upper().str.contains('DOG PARK'),0,1)
facility_polygon['Park'] = np.where(facility_polygon['Name'].str.upper().str.contains('PARK'),1,0)
facility_polygon = facility_polygon.sort_values(by=['Address','Rec Center','Not a Dog Park','Park'],ascending=False).drop_duplicates(subset='Address').copy()
facility_polygon = facility_polygon.set_index('OBJECTID')
facility_polygon = facility_polygon.drop(['CD_Real','Region','Name.1','Park_Class','TOOLTIP',
                                          'Area_Acres','Not a Dog Park','Park'],axis=1).copy()

#set new columns that need to be filled out to null
for item in ['Capacity', 'ADA Compliant?', 'Current emergency shelter?',
       'Hours of Operation', 'Indoor/Outdoor', 'Extra considerations']:
    
    facility_polygon[item]= nan
    
facility_polygon = facility_polygon.sort_values(by='ACRES',ascending=False)
facility_polygon.to_csv('rec_park_facilities_cleaned.csv')

### Using alternate info

In [535]:
main_facilities_sheet.columns

Index(['Type', 'Name', 'Street', 'Zip', 'RAP Category', 'lon', 'lat',
       'Capacity', 'ADA Compliant?', 'Current emergency shelter?',
       'Hours of Operation', 'Indoor/Outdoor', 'Extra considerations',
       'Priority Category', 'Rec_Center', 'Street_No_Suff'],
      dtype='object')

In [437]:
#load sheet
main_facilities_sheet = pd.read_excel('rap_facilities_libraries.xlsm')

#remap RAP categories to sort and drop duplicates
main_facilities_sheet['Priority Category'] = main_facilities_sheet['RAP Category'].map({'Recreational Centers':4,
                                           'Sport Facilities':3,
                                           'Parks':2,'Aquatics':1})
                                                                                      

#look for whether "Rec Center" is in the title and create column based on this
main_facilities_sheet['Rec_Center'] = np.where(main_facilities_sheet['Name'].str.contains('Recreation Center'),1,0)

#strip out street suffixes

street_drop = ['Ave', 'Street', 'St', 'Blvd', 'Dr', 'Road', 'Place', 'Pl', 'Rd',
       'Avenue', 'Drive', 'Way',  'Walk']

main_facilities_sheet['Street_No_Suff'] = np.where(main_facilities_sheet['Street'].str.contains('|'.join(street_drop)),main_facilities_sheet['Street'].str.strip('|'.join(street_drop)),
         main_facilities_sheet['Street'])

main_facilities_sheet['Street_No_Suff'] = main_facilities_sheet['Street_No_Suff'].str.strip()

#main_facilities_sheet = main_facilities_sheet.sort_values(by=['Street_No_Suff','Rec_Center','Priority Category'],ascending=False).drop_duplicates(subset='Street_No_Suff')
#main_facilities_sheet = main_facilities_sheet.reset_index(drop=True)


#possible names for matching
name_bucket = facility_polygon['Name'].values

#create match
fuzzy_match_rap = main_facilities_sheet['Name'].apply(lambda x: process.extractOne(x,name_bucket))

#convert to DF
fuzzy_match_rap = pd.DataFrame(list(fuzzy_match_rap.values))

#combine with main set
main_facilities_sheet = pd.merge(main_facilities_sheet,fuzzy_match_rap,left_index=True,right_index=True).rename(columns={0:'Fuzzy Facility',1:'Fuzzy Score'})

combined_facilities = pd.merge(main_facilities_sheet,facility_polygon,left_on='Fuzzy Facility',right_on='Name',how='left',suffixes=('','_x'))

#remove new features for score below 90
combined_facilities.loc[combined_facilities['Fuzzy Score']<90, 
                        ['Park_ID','CD_Real','Address','Region','Park_Class','ACRES','Fuzzy Facility']] = nan
combined_facilities = combined_facilities.rename(columns={'Fuzzy Facility':'Name_Alternate'})

combined_facilities = combined_facilities.drop(['Street_No_Suff','OBJECTID','Name_x','Name.1','CD_NEW','Fuzzy Score','TOOLTIP','Area_Acres'],axis=1)

cols = list(combined_facilities.columns)
for item in ['Name','Name_Alternate']:
    cols.remove(item)
combined_facilities = combined_facilities[['Name','Name_Alternate']+cols]

combined_facilities.to_csv('RAP_facilities.csv')